In [1]:
%matplotlib inline
import pandas as pd

import numpy as np

import seaborn as sns
import scipy as sci
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from numpy import array
from numpy import argmax
import keras as ker
from sklearn.ensemble import RandomForestRegressor

/local/java/python-pip-packages/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('/dcs/17/u1403100/Downloads/train_processed.csv')

In [9]:
np.clip(y_test,0.01,10**6)

array([ 12.,  18.,  16., ...,  18.,  20.,  22.])

In [11]:


def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    y_pred=np.clip(y_pred,0.01,10**6)
    y=np.clip(y,0.01,10**6)
    to_sum = [(np.log(y_pred[i] + 1)) - (np.log( y[i] + 1) ** 2.0) for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [13]:
def rmse(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(y_pred[i]-y[i]) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [4]:
dictionary=df[['brand_name','price']].groupby('brand_name').mean()
di=dictionary.to_dict()['price']
df['brand_name']=df.loc[:,'brand_name'].replace(di)

In [5]:
dictionary=df[['category3','price']].groupby('category3').mean()
di=dictionary.to_dict()['price']
df['category3']=df.loc[:,'category3'].replace(di)

In [6]:

df.loc[:, df.dtypes == np.int64]=df.loc[:, df.dtypes == np.int64].astype('int16')

In [7]:
#df[['category2','category3','brand_name']]= df[['category2','category3','brand_name']].astype('category')
df_x=df.drop(['name','item_description','category_name','category','category2' ,'price','brand_name'],axis=1).values
df_y=df['price'].values

In [ ]:
#category2 = pd.get_dummies(df['category2']).values
#category3 = pd.get_dummies(df['category3']).values

In [ ]:
#df_x=np.concatenate([df_x.values,category3],axis=1)

In [ ]:
i=0
df2 = pd.DataFrame(np.zeros((1,10)) ) 
for k in range(5):
    kf = KFold(n_splits=4)
    for train_index, test_index in kf.split(df):
        result=[]
        X_train, X_test = df_x[train_index], df_x[test_index]
        y_train, y_test = df_y[train_index], df_y[test_index]

        neigh = linear_model.Lasso(alpha=0.05*(k+1))
        neigh.fit(X_train,y_train)
        err=rmse(neigh.predict(X_test),y_test)
        result.append(err)
        
        
    df2.iloc[0,k]=np.average(result)
print(df2)

In [ ]:
i=0
df2 = pd.DataFrame(np.zeros((5,5)) )
hidden_layer_sizes=[(20,),(40,),(20,20),(40,20) ,(20,20,20)]
for i in range(5):
    for k in range(5):
        kf = KFold(n_splits=4)
        for train_index, test_index in kf.split(df):
            result=[]
            X_train, X_test = df_x[train_index], df_x[test_index]
            y_train, y_test = df_y[train_index], df_y[test_index]

            neigh = MLPRegressor(alpha=0.05*(k+1),hidden_layer_sizes=hidden_layer_sizes[i])
            neigh.fit(X_train,y_train)
            err=rmse(neigh.predict(X_test),y_test)
            result.append(err)
            print(err)


        df2.iloc[i,k]=np.average(result)
        
print(df2)



In [ ]:
n_estimators=[5,10,20,30,50]
max_depth=[10, 20, 40, 70, None]
i=0
df2 = pd.DataFrame(np.zeros((5,10)) ) 
for i in range(5):
    for k in range(5):
        kf = KFold(n_splits=4)
        for train_index, test_index in kf.split(df):
            result=[]
            X_train, X_test = df_x[train_index], df_x[test_index]
            y_train, y_test = df_y[train_index], df_y[test_index]

            neigh = RandomForestRegressor(random_state = 42,n_estimators=n_estimators[i],max_depth=max_depth[k])
            neigh.fit(X_train,y_train)
            err=rmse(neigh.predict(X_test),y_test)
            result.append(err)
            print(err)
        
        df2.iloc[i,k]=np.average(result)
        
    
print(df2)